บทเรียนนี้จะเน้นการจัดการข้อมูลหลายชุด (Multiple Data Sources) ซึ่งเป็นงานจริงที่ป๋าต้องเจอแน่นอน เช่น การเอารายงานหลายเดือนมารวมกัน หรือการดึงชื่อสินค้าจากตาราง Master มาแปะ

In [6]:
import pandas as pd
import numpy as np
import glob


In [ ]:

# --- 1. สร้างไพล์ยอดขายรายเดือน (สำหรับโจทย์รวมไฟล์) ---
# เดือนมกราคม
df_jan = pd.DataFrame({
    'Order_ID': ['J001', 'J002', 'J003'],
    'Product_Code': ['P1', 'P2', 'P1'],
    'Qty': [10, 5, 20]
})
df_jan.to_csv('../datasets/multiple_data/sales_jan.csv', index=False)

# เดือนกุมภาพันธ์
df_feb = pd.DataFrame({
    'Order_ID': ['F001', 'F002'],
    'Product_Code': ['P3', 'P1'],
    'Qty': [8, 15]
})
df_feb.to_csv("../datasets/multiple_data/sales_feb.csv", index=False)

# --- 2. สร้างไฟล์ Master สินค้า (สำหรับโจทย์ VLOOKUP) ---
df_master = pd.DataFrame({
    'P_Code': ['P1', 'P2', 'P3', 'P4'],
    'P_Name': ['ปากกา', 'ดินสอ', 'ยางลบ', 'ไม้บรรทัด'],
    'Price': [10, 5, 3, 12]
})
df_master.to_csv('../datasets/multiple_data/product_master.csv', index=False)

# --- 3. สร้างไฟล์เรทราคาน้ำหนัก (สำหรับโจทย์ XLOOKUP - Approximate Match) ---
# น้ำหนักไม่เกินนี้ คิดราคานี้ (Tier ราคา)
df_rates = pd.DataFrame({
    'Max_Weight': [1, 5, 10, 20],
    'Shipping_Cost': [30, 50, 80, 150]
})
df_rates.to_csv('../datasets/multiple_data/shipping_Cost.csv', index=False)

print('สร้างไฟล์จำลองเสร็จเรียบร้อย')

1. รวมไฟล์ Excel/CSV หลายไฟล์เป็นอันเดียว (Concatenation)
โจทย์: ฝ่ายขายส่งไฟล์ยอดขายแยกเดือนมา ป๋าต้องรวมเป็นก้อนเดียวเพื่อทำกราฟปี



แบบง่าย (Easy Mode)
ถ้ารู้ชื่อไฟล์แน่นอน และมีแค่ไม่กี่ไฟล์ จับมาใส่ List แล้วรวมเลยครับ

In [ ]:
df1 = pd.read_csv('../datasets/multiple_data/sales_jan.csv')
df2 = pd.read_csv('../datasets/multiple_data/sales_feb.csv')

# รวมร่างด้วย pd.concat (เอามาต่อท้าย)
all_sales = pd.concat([df1, df2], ignore_index=True)

print(df1, '\n')
print(df2, '\n')
print(all_sales)

  Order_ID Product_Code  Qty
0     J001           P1   10
1     J002           P2    5
2     J003           P1   20 

  Order_ID Product_Code  Qty
0     F001           P3    8
1     F002           P1   15 

  Order_ID Product_Code  Qty
0     J001           P1   10
1     J002           P2    5
2     J003           P1   20
3     F001           P3    8
4     F002           P1   15


แบบยาก (Pro Mode: Automate)
สถานการณ์จริง: ถ้าป๋ามีไฟล์ 12 เดือน หรือ 100 ไฟล์ จะมานั่งพิมพ์ชื่อไฟล์ทีละบรรทัดไม่ไหว และป๋าต้องการรู้ด้วยว่า "ข้อมูลนี้มาจากไฟล์ไหน?" (Source Identification)

เราจะใช้ไลบรารี glob มาช่วยกวาดชื่อไฟล์ทั้งหมดในโฟลเดอร์ครับ

In [23]:
# 1. หาไฟล์ทั้งหมดที่ขึ้นต้นด้วย sales_ และลงท้ายด้วย .csv

file_list = glob.glob('../datasets/multiple_data/sales_*.csv')

all_data = [] # สร้าง list ว่างไว้รอเก็บ
for filename in file_list:
    df = pd.read_csv(filename)
    df['Source_File'] = filename        # เทคนิคสำคัญ * สร้างคอลัมน์บอกที่มา
    all_data.append(df)

# รวมครั้งเดียว
final_df = pd.concat(all_data, ignore_index=True)

print('--- รวมแบบ Pro (รู้ที่มา) ---')
final_df.to_excel('../output/concat_sales.xlsx')
print(final_df.head())

--- รวมแบบ Pro (รู้ที่มา) ---
  Order_ID Product_Code  Qty                              Source_File
0     F001           P3    8  ../datasets/multiple_data\sales_feb.csv
1     F002           P1   15  ../datasets/multiple_data\sales_feb.csv
2     J001           P1   10  ../datasets/multiple_data\sales_jan.csv
3     J002           P2    5  ../datasets/multiple_data\sales_jan.csv
4     J003           P1   20  ../datasets/multiple_data\sales_jan.csv
